# Deploy COA Objects in Transcend:   DBInfo (Demo Project)

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%connect name=Transcend

In [ ]:
call adlste_coa.sp_meta_objects_refresh('DBCInfo') 

## Stage Object (adlste_coa_stg)

In [ ]:
drop table adlste_coa_stg.stg_dat_dbcinfo;

In [ ]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

In [ ]:
select count(*) from adlste_coa_stg.stg_dat_dbcinfo;

## Target Table and View (adlste_coa)

In [ ]:
DROP TABLE adlste_coa.coat_dat_DBCinfo;

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, LogDate        DATE Default Current_Date
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) Primary Index(Site_ID, LogDate);

In [ ]:
REPLACE VIEW adlste_coa.coa_dat_DBCinfo
as locking row for access
Select * from adlste_coa.coat_dat_DBCinfo;

## Stored Proc to manage the merge

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_dbcinfo ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_dbcinfo';
    SET callingsp = 'adlste_coa.sp_dat_dbcinfo';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,current_date as StartDate
    ,current_date as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_dbcinfo 
    Group by  1,2,3
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START') ;

    DELETE FROM adlste_coa.coat_dat_dbcinfo WHERE (Site_ID, LogDate) IN
    (Select Site_ID, Current_Date From adlste_coa_stg.stg_dat_dbcinfo );
    
    Insert into adlste_coa.coat_dat_dbcinfo 
    Select Account_Name, Site_ID, Current_Date, InfoKey, InfoData 
    from adlste_coa_stg.stg_dat_dbcinfo
    group  by 1,2,3,4,5; 
    
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END') ;

END;

In [ ]:
-- Generate Test Data
insert into adlste_coa_stg.stg_dat_DBCinfo
select 'test account', 'testsiteid123', infokey, infodata from dbc.dbcinfo;

In [ ]:
call adlste_coa.sp_dat_dbcinfo('1.1')

In [ ]:
select * from adlste_coa.coa_audit_runlog order by 1

In [ ]:
select * from adlste_coa.coa_audit_summary order by 1

## Test the Process

In [ ]:
insert into adlste_coa_stg.stg_dat_dbcinfo 
Select 'Test_Account',  'Test_SiteID', InfoKey, InfoData from dbc.dbcinfo;

Select * from adlste_coa_stg.stg_dat_dbcinfo 

In [ ]:
call adlste_coa.sp_dat_dbcinfo('1.1');

In [ ]:
Select * from adlste_coa.coa_dat_dbcinfo 